## Solution to the reaction-diffusion profile around a Janus particle

In this notebook, I compute the solution of the reaction-diffusion

$$(\nabla^2-\kappa^2) f = 0$$

with a reactive boundary condition on a sphere centered at the origin
of the coordinate system, with azimuthal symmetry.

In general, one can write these solutions as

$$c = \sum_{l=0}^\infty a_l \frac{\sqrt{\kappa r_0}}{K_{l+1/2}(\kappa r_0)} \frac{K_{l+1/2}(\kappa r)}{\sqrt{\kappa r}} P_l(\cos\theta)$$

Here, the problem is specified by having only the top half of the sphere catalytic.

The $K_{l+1/2}$ are the modified Bessel function or half-integer order.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import special
from scipy import integrate
import itertools

π = np.pi

### Getting the Legendre polynomials

In [ ]:
# Define a range of 
mu_r = np.linspace(-1, 1, 101)

In [ ]:
[plt.plot(mu_r, special.legendre(i)(mu_r), label=r'$P_%i(\mu)$' % i) for i in range(6)];
plt.xlabel(r'$\mu$')
plt.ylabel(r'$P_l(\mu)$')
plt.legend()

### Getting the Bessel function

In [ ]:
r_range = np.linspace(1, 8, 101)

[plt.plot(r_range, special.kv(i+0.5, r_range)/np.sqrt(r_range)/special.kv(i+0.5, r_range[0]), label=r'$K_{'+str(i)+r'+1/2}(r)$') for i in [0, 1, 2, 3]]

plt.xlabel(r'$r$')
plt.ylabel(r'$\frac{K_{l+1/2}(r)}{\sqrt{r}}$')
plt.legend()

## Definition of the problem

Having written the solution as

$$\sum_l M_{jl} a_l = b_j$$

We must compute $M_{jl}$ and $b_j$

$$m_jl = \frac{k_0}{k_D}  \int_0^1 d\mu\ P_l(\mu) P_j(\mu) + r_0 \kappa \left[ \frac{K_{l+3/2}(\kappa r_0)}{K_{l+1/2}(\kappa r_0)} -\frac{l}{\kappa r_0}\right] \frac{2}{2l+1}\delta_{l,j}$$


### Compute integrals of Legendre polynomials

Here, we compute $\int_0^1 P_l(\mu) P_j(\mu) d\mu$ and $\int_0^1 P_j(\mu) d\mu$

In [ ]:
def legendre_int_lj(l, j):
    Pl = special.legendre(l)
    Pj = special.legendre(j)
    sol, err = integrate.quad(lambda mu: Pl(mu)*Pj(mu), 0, 1)
    return sol

def legendre_int_j(j):
    Pj = special.legendre(j)
    sol, err = integrate.quad(lambda mu: Pj(mu), 0, 1)
    return sol

### Compute the Bessel term

$$\frac{K_{l+3/2}(\kappa r_0)}{K_{l+1/2}(\kappa r_0)}$$

In [ ]:
def bessel_term(l, kr0):
    return special.kv(l+1.5, kr0)/special.kv(l+0.5,kr0)

### Build the left-hand side matrix

In [ ]:
Ncut = 40

M = np.zeros((Ncut, Ncut))
delta = np.eye(Ncut)

T = 1
m = 1
p = 1
R = 3.5
D = 1.17
kD = 4*π*R*D
k0 = p*R**2*np.sqrt(8*π*T/m)
k2 = 0.02
κ = np.sqrt(k2/D)
r0 = R

for i, j in itertools.product(range(Ncut), range(Ncut)):
    T1 = k0/kD * legendre_int_lj(i, j)
    T2 = (r0*κ*bessel_term(j, κ*r0) - j)*2/(2*j+1)*delta[i,j]
    M[i,j] = T1+T2
    

### Build the right-hand side vector

In [ ]:
b = k0/kD * np.array([legendre_int_j(j) for j in range(Ncut)])

### Solve

In [ ]:
a = np.linalg.solve(M, b)

In [ ]:
# Verification

assert np.allclose(np.dot(M, a), b)

### Define the solution $c(r, \mu)$

In [ ]:
def c(r, mu):
    res = 0
    for i in range(Ncut):
        res += a[i]*special.legendre(i)(mu)*special.kv(i+0.5, κ*r)/np.sqrt(κ*r)/(special.kv(i+0.5, κ*R)/np.sqrt(κ*R))
    return res

In [ ]:
# Compute the solution on a grid

MU = np.linspace(-1, 1, 64)
RR = np.linspace(R, 6.9*R/2.9, 64)

data = [[c(r, mu) for r in RR] for mu in MU]
data = np.array(data)

# Convert to cartesian coordinates for display
X = RR.reshape((1,-1)) * np.sqrt(1-MU**2).reshape((-1, 1))
Z = RR.reshape((1,-1)) * MU.reshape((-1, 1))


In [ ]:
plt.figure(figsize=(8,10))
plt.subplot(111, aspect=1)
plt.pcolormesh(X, Z, data)
plt.colorbar()
plt.contour(X, Z, data, colors='k')